Data set is Movie Lens Data set downloaded from Kaggle.
This data set has 4 CSVs with attributes resting in different CSVs all of them having 1 or other columns in common.

```Tasks:```
    1. Look at all the CSVs
    2. Figure out the common columns
    3. Make a common DataFrame out of all such that all the information gets centered in one base.
    4. Check for nulls if any in the data base and treat them accordingly
    5. Interesting thing is all the genres have multiple phrases seperated by pipe. We need to assign them only one. Do so but 
        in a random fashion
    6. Finally once the dataset is ready answer following questions:
        a. Which are the top 10 movies with highest rating
        b. Which are the top 10 genres with highest rating
        c. Which are the top most tags getting featured ?
            
Let us get all the CSVs in to DFs in below global import section        

In [1]:
%matplotlib inline
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from numpy.random import randn
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta 
from functools import reduce
import random
# 
links = pd.read_csv('D:\Study Materials\Documents to play with in python\Movie Lens\links.csv',error_bad_lines=False) 
movies = pd.read_csv('D:\Study Materials\Documents to play with in python\Movie Lens\movies.csv',error_bad_lines=False) 
rating = pd.read_csv(r'D:\Study Materials\Documents to play with in python\Movie Lens\ratings.csv',error_bad_lines=False) 
tags = pd.read_csv(r'D:\Study Materials\Documents to play with in python\Movie Lens\tags.csv',error_bad_lines=False) 

Let us take a look at the individual Dataframes

In [2]:
links.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [3]:
movies.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


Let us now clean up the genres column of movies dataframe
Get only one out of the many pipe seperated genres against each movieid

Steps to get a random genre out of all pipe seperated genres for each film

1. Get the count of genres against a film. It will be count(pipes) + 1
2. Get a random number between 0 and (count of genres obtained above)
3. Split the genre column by pipe symbol and get the resultant list of genres against a movie
4. Out of that list assign the movie a genre that is in the list indexed at a number obtained in step 2

Below steps does the trick in just one go

In [4]:
movies['genres'] = movies['genres'].apply(lambda x: x.split('|')[np.random.randint(0,(x.count('|')+1))])
movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
3,4,Waiting to Exhale (1995),Drama
4,5,Father of the Bride Part II (1995),Comedy


Unique genres we have are

In [5]:
movies['genres'].unique()

array(['Adventure', 'Fantasy', 'Comedy', 'Drama', 'Action', 'Romance',
       'Children', 'Horror', 'Crime', 'Thriller', 'Mystery', 'Animation',
       'Sci-Fi', 'Documentary', 'IMAX', 'War', 'Musical', 'Western',
       'Film-Noir', '(no genres listed)'], dtype=object)

In [6]:
rating.drop_duplicates()
rating.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


In [7]:
tags.drop_duplicates()
tags.head(2)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996


Now let us make one single data frame.
Every table has movieid column common to them hence that will become our joining key.
Let us form a consolidated data frame out of all.

In [8]:
dataframe_1 = pd.merge(movies,links,on = ['movieId','movieId'])
dataframe_2 = pd.merge(rating,tags,left_on = ['userId','movieId'],right_on = ['userId','movieId'],validate = 'one_to_many',how = 'outer')
final_dataframe = pd.merge(dataframe_1,dataframe_2,on = ['movieId','movieId'])
final_dataframe.fillna(0)
final_dataframe.head(3)

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp_x,tag,timestamp_y
0,1,Toy Story (1995),Adventure,114709,862.0,1,4.0,9.649827e+08,NaN,NaN
1,1,Toy Story (1995),Adventure,114709,862.0,5,4.0,8.474350e+08,NaN,NaN
2,1,Toy Story (1995),Adventure,114709,862.0,7,4.5,1.106636e+09,NaN,NaN


In [9]:
#Checking for counts in the source and destination data frame
print(movies.count())
print(links.count())
print(dataframe_1.count())
print(tags.count())
print(rating.count())
print(dataframe_2.count())
print(final_dataframe.count())

movieId    9742
title      9742
genres     9742
dtype: int64
movieId    9742
imdbId     9742
tmdbId     9734
dtype: int64
movieId    9742
title      9742
genres     9742
imdbId     9742
tmdbId     9734
dtype: int64
userId       3683
movieId      3683
tag          3683
timestamp    3683
dtype: int64
userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64
userId         102884
movieId        102884
rating         102677
timestamp_x    102677
tag              3683
timestamp_y      3683
dtype: int64
movieId        102884
title          102884
genres         102884
imdbId         102884
tmdbId         102871
userId         102884
rating         102677
timestamp_x    102677
tag              3683
timestamp_y      3683
dtype: int64


Finally let us answer the simple questions as asked

In [10]:
# Answer 1

top_10_movies_df = final_dataframe['rating'].groupby(final_dataframe['title']).mean().sort_values(ascending = False).head(10)
DataFrame(top_10_movies_df)

,rating
title,
I'm the One That I Want (2000),5.0
Vacations in Prostokvashino (1980),5.0
My Love (2006),5.0
Cherish (2002),5.0
Paper Birds (Pájaros de papel) (2010),5.0
All the Vermeers in New York (1990),5.0
Lumberjack Man (2015),5.0
Last Hurrah for Chivalry (Hao xia) (1979),5.0
Cheburashka (1971),5.0


In [11]:
#Answer 2

top_10_genres_df = final_dataframe['rating'].groupby(final_dataframe['genres']).mean().sort_values(ascending = False).head(10)
DataFrame(top_10_genres_df)

,rating
genres,
Film-Noir,3.913793
Documentary,3.829609
War,3.740521
IMAX,3.720946
Drama,3.701621
Crime,3.686984
Mystery,3.629032
Western,3.550198
Animation,3.550125


In [12]:
#Answer 3

top_10_tags_df = final_dataframe['rating'].groupby(final_dataframe['tag']).mean().sort_values(ascending = False).head(10)
DataFrame(top_10_tags_df)

,rating
tag,
innovative,5.0
Missionary,5.0
Funny,5.0
French,5.0
Francis Ford Coppola,5.0
ocean,5.0
classic sci-fi,5.0
offensive,5.0
classic movie,5.0
